In [11]:
# pip install optuna captum

In [12]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, Dataset
from captum.attr import IntegratedGradients

torch.manual_seed(42)
np.random.seed(42)

In [13]:
def load_data(file_path):
    """加载训练数据并返回DataFrame和日期列"""
    df = pd.read_csv(file_path)
    print(f"原始数据维度: {df.shape}")  # 输出数据的行数和列数（如：(442, 4000)）

    # 转换日期列并验证
    dates = pd.to_datetime(df.columns[1:], format='%d/%m/%Y')
    print(f"日期列数量: {len(dates)}")  # 输出日期列的数量（如：4000天）

    return df, dates

In [14]:
def preprocess_data(df, window_size=10):
    scaled_data = {}
    for company_id in df['ID'].unique():
        # 提取单个公司的数据（跳过ID列）
        company_series = df[df['ID'] == company_id].iloc[:, 1:].values.astype(float)
        # 标准化（保持二维）
        scaler = StandardScaler()
        scaled = scaler.fit_transform(company_series.reshape(-1, 1))  # 形状为 (n_samples, 1)
        scaled_data[company_id] = {
            'data': scaled,
            'scaler': scaler
        }
        # print(f"公司 {company_id} 标准化后的数据维度: {scaled.shape}")
    # print(scaled_data)
    print(f"标准化后的数据维度: {len(scaled_data)}")  # 输出公司数
    return scaled_data

In [15]:
# =====================================================================================
# 2. 模型定义
# =====================================================================================
class LSTMModel(nn.Module):
    """LSTM模型实现"""
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=2, output_dim=1):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # LSTM层
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )

        # 全连接层
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        # 初始化隐藏状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # LSTM前向传播
        out, _ = self.lstm(x, (h0, c0))
        # 取最后一个时间步的输出
        out = self.fc(out[:, -1, :])
        return out

In [16]:
# =====================================================================================
# 3. 数据集与数据加载器
# =====================================================================================
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

    def __len__(self):
        return max(0, len(self.data) - self.window_size)

    def __getitem__(self, idx):
        features = self.data[idx:idx + self.window_size]  # 形状为 (window_size, 1)
        target = self.data[idx + self.window_size, 0]  # 标量

        # 转换为张量
        features = torch.tensor(features, dtype=torch.float)  # 形状为 (window_size, 1)
        target = torch.tensor([target], dtype=torch.float)  # 包裹成 [scalar] 形状

        return features, target

In [17]:
# =====================================================================================
# 4. 训练与验证函数
# =====================================================================================
def train_model(model, dataloader, criterion, optimizer, device):
    """模型训练函数"""
    model.train()
    total_loss = 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y.view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


In [18]:
def validate_model(model, dataloader, criterion, device):
    """模型验证函数"""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            loss = criterion(outputs, y.view(-1, 1))
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [19]:
import copy
# =====================================================================================
# 5. Optuna超参数优化
# =====================================================================================
import time
def objective(trial, window_size, train_datasets, val_datasets):
    # 超参数搜索空间
    best_model_state = None  # ✅ 确保变量在函数内总是有值
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    hidden_dim = trial.suggest_int('hidden_dim', 16, 64)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    num_layers = trial.suggest_int('num_layers', 1, 2)
    patience = trial.suggest_int("patience", 3, 10)

    # init model
    device = torch.device("mps" if torch.mps.is_available() else "cpu")
    model = LSTMModel(input_dim=1, hidden_dim=hidden_dim, num_layers=num_layers).to(device)
    criterion = nn.MSELoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
                     for dataset in train_datasets]
    val_loaders = [DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
                   for dataset in val_datasets]

    best_loss = float('inf')
    early_stop_counter = 0



    for epoch in range(10):  # 减少总epoch数
        model.train() # train 的很快
        start_time = time.time()

        """
        下面这个部分是最慢的
        """
        for loader in train_loaders:
            for X, y in loader:
                # X = X.unsqueeze(-1).to(device)  # 确保形状 (B × window_size × 1)
                X = X.to(device, non_blocking=True)  # 确保X的形状为 (batch_size × window_size × 1)
                # print("X.shape is ", X.shape)
                y = y.to(device, non_blocking=True)
                # y = y.squeeze(-1)  # 现在再去掉多余维度，确保形状一致

                optimizer.zero_grad()
                outputs = model(X)
                loss = criterion(outputs, y)
                loss.backward()
                optimizer.step()
        end_time = time.time()


        print(f"epoch_{epoch}遍历dataloader用时{(end_time-start_time)}" )
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for loader in val_loaders:
                for X, y in loader:
                    X = X.to(device, non_blocking=True)
                    # X = X.unsqueeze(-1).to(device)
                    y = y.to(device, non_blocking=True)
                    outputs = model(X)
                    val_loss += criterion(outputs, y).item()
        avg_val_loss = val_loss / len(val_loaders)
        print(f"epoch_{epoch}eval后遍历dataloader用时{(time.time()-end_time)}" )

        # 6. 更新最优模型
        if avg_val_loss < best_loss:
            print(f"有更新, 最新的loss是{avg_val_loss}")
            best_loss = avg_val_loss
            best_model_state = copy.deepcopy(model.state_dict())  # ✅ 彻底复制参数，避免 shallow copy 问题
            trial.set_user_attr("best_model", best_model_state)
            print("是否真的存进去了: ", "best_model" in trial.study.user_attrs)
            # 手动保存到文件，防止 Optuna 多进程问题
            torch.save(best_model_state, "best_model.pth")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                break  # 早停

        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # 7. 存储最优模型到 Optuna
    if best_model_state is not None:
        trial.set_user_attr("best_model", best_model_state)  # ✅ 再次确保写入 Optuna

    return best_loss

In [20]:
# =====================================================================================
# 6. 主函数
# =====================================================================================

# 1. 加载数据
df, dates = load_data('playing-the-stock-market-is-a-fools-game/train.csv')
scaled_data = preprocess_data(df, window_size=10)

# 2. 超参数设置
window_size = 10
test_ratio = 0.2

# 3. 数据准备
train_datasets = []
val_datasets = []
for company_id in scaled_data.keys():
    data = scaled_data[company_id]['data']
    # print(data)
    split = int(len(data) * (1 - test_ratio))

    # 确保训练集和验证集至少有 window_size 的数据
    train_data = data[:split]
    val_data = data[max(split - window_size, 0):]  # 确保验证集长度足够

    train_datasets.append(TimeSeriesDataset(train_data, window_size))
    val_datasets.append(TimeSeriesDataset(val_data, window_size))

# 4. Optuna优化
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, window_size, train_datasets, val_datasets),
              n_trials=20)

# 5. 使用最佳模型预测
best_params = study.best_params
device = torch.device("mps" if torch.mps.is_available() else "cpu")
best_model = LSTMModel(
    input_dim=1,
    hidden_dim=best_params['hidden_dim'],
    num_layers=best_params['num_layers']
).to(device)
try:
    best_model.load_state_dict(study.user_attrs['best_model'])
except:
    best_model_state = torch.load("best_model.pth")

# 6. 预测
submission = pd.read_csv('playing-the-stock-market-is-a-fools-game/sample_submission.csv')
for idx, row in submission.iterrows():
    company_id = row['ID']
    # 获取标准化后的数据
    company_info = scaled_data[company_id]
    last_window = company_info['data'][-window_size:].reshape(1, window_size, 1)
    # 预测
    with torch.no_grad():
        input_tensor = torch.tensor(last_window, dtype=torch.float).to(device)
        pred = best_model(input_tensor)
    # 反标准化
    pred_value = company_info['scaler'].inverse_transform(pred.cpu().numpy())[0][0]
    submission.at[idx, 'value'] = pred_value

# 7. 保存结果
submission.to_csv('submission.csv', index=False)

# 8. 模型解释（可选）
# 使用Captum分析特征重要性
inputs = torch.tensor(last_window, dtype=torch.float32, requires_grad=True).to(device)
ig = IntegratedGradients(best_model)
attributions, delta = ig.attribute(inputs, target=0, return_convergence_delta=True)
# 绘制特征重要性图
plt.figure(figsize=(10,5))
plt.bar(range(window_size), attributions[0,:,0].cpu().detach().numpy())
plt.title('Feature Importance (Last Window)')
plt.xlabel('Time Step')
plt.ylabel('Attribution Score')
plt.savefig('feature_importance.png')


原始数据维度: (442, 3022)
日期列数量: 3021


[I 2025-03-26 14:52:50,862] A new study created in memory with name: no-name-09545f94-862d-4761-a1ee-951d6bd31e45


标准化后的数据维度: 442
epoch_0遍历dataloader用时54.76955795288086
epoch_0eval后遍历dataloader用时6.445476055145264
epoch_1遍历dataloader用时56.347148180007935
epoch_1eval后遍历dataloader用时6.295850992202759
epoch_2遍历dataloader用时57.66811919212341
epoch_2eval后遍历dataloader用时6.6984968185424805
epoch_3遍历dataloader用时49.112061977386475
epoch_3eval后遍历dataloader用时6.256008863449097
epoch_4遍历dataloader用时61.128135204315186
epoch_4eval后遍历dataloader用时6.172161817550659
epoch_5遍历dataloader用时60.44142174720764
epoch_5eval后遍历dataloader用时6.110482215881348
epoch_6遍历dataloader用时60.929561138153076


[I 2025-03-26 15:00:15,508] Trial 0 finished with value: inf and parameters: {'optimizer': 'SGD', 'lr': 4.0170994388202094e-05, 'hidden_dim': 52, 'batch_size': 46, 'num_layers': 1, 'patience': 7}. Best is trial 0 with value: inf.


epoch_6eval后遍历dataloader用时6.2581870555877686
epoch_0遍历dataloader用时45.7811758518219
epoch_0eval后遍历dataloader用时6.494537115097046
epoch_1遍历dataloader用时46.29562997817993
epoch_1eval后遍历dataloader用时6.669595956802368
epoch_2遍历dataloader用时45.60125803947449


...

epoch_0eval后遍历dataloader用时10.903244972229004
有更新, 最新的loss是48.13574231674313
是否真的存进去了:  False
epoch_0遍历dataloader用时39.94445204734802


[I 2025-03-26 16:50:07,721] Trial 19 pruned. 


epoch_0eval后遍历dataloader用时5.5163938999176025


/var/folders/m1/dz6zsv052q319bqsbfdq06qm0000gn/T/ipykernel_88852/3032149754.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model_state = torch.load("best_model.pt

TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.